In [9]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np
from datetime import datetime
from config import password, login, server
import time

In [2]:
mt5.initialize(login=login, server=server, password=password)

True

In [76]:
# settings
symbol = 'USDJPY'
timeframe = mt5.TIMEFRAME_M5
start_pos = 0
num_bars = 1000

fsma_period = 14
ssma_period = 28

In [77]:
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)
df = pd.DataFrame(bars)[['time', 'open', 'close', 'low', 'high']]
df['time'] = pd.to_datetime(df['time'], unit='s')
df['slow_sma'] = df['close'].rolling(ssma_period).mean()
df['fast_sma'] = df['close'].rolling(fsma_period).mean()

df.dropna(inplace=True)
df

,time,open,close,low,high,slow_sma,fast_sma
27,2023-10-03 15:45:00,149.879,149.893,149.874,149.933,149.929429,149.913857
28,2023-10-03 15:50:00,149.889,149.884,149.866,149.895,149.927321,149.910000
29,2023-10-03 15:55:00,149.881,149.881,149.853,149.894,149.924857,149.906071
30,2023-10-03 16:00:00,149.887,150.048,149.871,150.064,149.928750,149.914357
31,2023-10-03 16:05:00,150.049,150.083,150.029,150.154,149.933071,149.926429
...,...,...,...,...,...,...,...
995,2023-10-09 00:25:00,149.180,149.139,149.139,149.206,149.176179,149.109714
996,2023-10-09 00:30:00,149.140,149.151,149.138,149.196,149.170071,149.113786
997,2023-10-09 00:35:00,149.147,149.156,149.125,149.178,149.163714,149.118714
998,2023-10-09 00:40:00,149.157,149.222,149.154,149.228,149.160107,149.126500


In [68]:
# Finding Crossovers
df['prev_fast_sma'] = df['fast_sma'].shift(1)

def find_crossovers(slow_sma, fast_sma, prev_fast_sma):
    if slow_sma < fast_sma and prev_fast_sma < slow_sma:
        return 'bull'
    elif slow_sma > fast_sma and prev_fast_sma > slow_sma:
        return 'bear'
    else:
        return None
    
df['crossover'] = np.vectorize(find_crossovers)(df['slow_sma'], df['fast_sma'], df['prev_fast_sma'])

signal = df[df['crossover'] == 'bull'].copy()
signal

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning:

invalid value encountered in find_crossovers (vectorized)



,time,open,close,low,high,slow_sma,fast_sma,prev_fast_sma,crossover
64,2023-03-28 07:50:00,130.641,130.649,130.600,130.649,130.615571,130.616857,130.614143,bull
65,2023-03-28 07:55:00,130.651,130.691,130.651,130.714,130.619429,130.624143,130.616857,bull
66,2023-03-28 08:00:00,130.695,130.739,130.695,130.797,130.624500,130.633143,130.624143,bull
159,2023-03-28 15:45:00,130.933,131.038,130.921,131.060,130.784536,130.792000,130.752643,bull
197,2023-03-28 18:55:00,130.959,130.955,130.938,130.967,130.980821,130.980929,130.978214,bull
...,...,...,...,...,...,...,...,...,...
39738,2023-10-06 02:15:00,148.505,148.549,148.493,148.549,148.452893,148.455714,148.447429,bull
39739,2023-10-06 02:20:00,148.545,148.574,148.536,148.582,148.457357,148.467357,148.455714,bull
39775,2023-10-06 05:20:00,148.637,148.621,148.619,148.650,148.579571,148.587143,148.578429,bull
39885,2023-10-06 14:30:00,148.962,149.376,148.907,149.409,149.052679,149.069929,149.047143,bull


In [ ]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
fig.show()

In [22]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
        return True
        
# logic
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.crossover == 'bear':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
            
            if data.crossover == 'bull':
                self.add_position(Position(data.time, data.close, 'buy', self.volume, 0, 0))
        
        return self.get_positions_df()

In [69]:
sma_crossover_strategy = Strategy(df, 10000, 100)
result = sma_crossover_strategy.run()

result

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2023-03-28 07:50:00,130.649,buy,100,0,0,2023-03-28 11:35:00,131.095,44.6,closed,10044.6
1,2023-03-28 07:55:00,130.691,buy,100,0,0,2023-03-28 11:35:00,131.095,40.4,closed,10085.0
2,2023-03-28 08:00:00,130.739,buy,100,0,0,2023-03-28 11:35:00,131.095,35.6,closed,10120.6
3,2023-03-28 15:45:00,131.038,buy,100,0,0,2023-03-28 17:35:00,130.906,-13.2,closed,10107.4
4,2023-03-28 18:55:00,130.955,buy,100,0,0,2023-03-28 19:00:00,130.857,-9.8,closed,10097.6
...,...,...,...,...,...,...,...,...,...,...,...
980,2023-10-06 02:15:00,148.549,buy,100,0,0,2023-10-06 04:15:00,148.508,-4.1,closed,11079.1
981,2023-10-06 02:20:00,148.574,buy,100,0,0,2023-10-06 04:15:00,148.508,-6.6,closed,11072.5
982,2023-10-06 05:20:00,148.621,buy,100,0,0,2023-10-06 10:00:00,148.873,25.2,closed,11097.7
983,2023-10-06 14:30:00,149.376,buy,100,0,0,2023-10-06 16:35:00,149.226,-15.0,closed,11082.7


In [70]:
px.line(result, x='close_datetime', y='pnl')

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [44]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
for i, row in result[result['status'] == 'closed'].iterrows():
    
    if row.profit > 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Green",width=3)
                     )
                      
    elif row.profit < 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Red",width=3)
                      )

    
fig.show()

c:\Users\sciensa\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

